In [1]:
import pandas as pd

from sklearn import ensemble
from sklearn.model_selection import cross_validate
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB,BernoulliNB,GaussianNB
import gzip

/home/canf/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
def loadData(df, scaler=None):
    data = pd.DataFrame(index=range(len(df)))
    
    data = df.get(['X','Y'])
    
    DayOfWeeks = df.DayOfWeek.unique()
    DayOfWeekMap = {}
    i = 0
    for day in DayOfWeeks:
        DayOfWeekMap[day] = i
        i += 1
    data = data.join(df['DayOfWeek'].map(DayOfWeekMap))
    
    PdDistricts = df.PdDistrict.unique()
    PdDistrictMap = {}
    i = 0
    for s in PdDistricts:
        PdDistrictMap[s] = i
        i += 1
    data = data.join(df['PdDistrict'].map(PdDistrictMap))
        
    date_time = pd.to_datetime(df.Dates)
    year = date_time.dt.year
    data['Year'] = year
    month = date_time.dt.month
    data['Month'] = month
    day = date_time.dt.day
    data['Day'] = day
    hour = date_time.dt.hour
    data['hour'] = hour
    minute = date_time.dt.minute
    time = hour*60+minute
    data['Time'] = time
    
    data['StreetCorner'] = df['Address'].str.contains('/').map(int)
    data['Block'] = df['Address'].str.contains('Block').map(int)
    
    X = data.values
    Y = None
    if 'Category' in df.columns:
        Y = df.Category.values
    
    return X, Y, scaler

In [3]:
def RFpredict(X,Y,Xhat):
    clf = ensemble.RandomForestClassifier()
    clf.set_params(min_samples_split=1000)
    clf.fit(X,Y)
    Yhat = clf.predict_proba(Xhat)
    return Yhat,clf

def NBpredict(X,Y,Xhat):
    clf = GaussianNB()
    clf.fit(X,Y)
    Yhat = clf.predict_proba(Xhat)
    return Yhat,clf

In [4]:
train = pd.read_csv("./input/train.csv")
X,Y,scaler = loadData(train)

test = pd.read_csv("./input/test.csv")
Xhat,_,__ = loadData(test)


In [5]:
print('-------------------------------------------------------------------------')
print(X)
print('-------------------------------------------------------------------------')
print(Y)
print('-------------------------------------------------------------------------')
print(Xhat)

-------------------------------------------------------------------------
[[-1.22425892e+02  3.77745986e+01  0.00000000e+00 ...  1.43300000e+03
   1.00000000e+00  0.00000000e+00]
 [-1.22425892e+02  3.77745986e+01  0.00000000e+00 ...  1.43300000e+03
   1.00000000e+00  0.00000000e+00]
 [-1.22424363e+02  3.78004143e+01  0.00000000e+00 ...  1.41300000e+03
   1.00000000e+00  0.00000000e+00]
 ...
 [-1.22403390e+02  3.77802656e+01  2.00000000e+00 ...  1.00000000e+00
   1.00000000e+00  0.00000000e+00]
 [-1.22390531e+02  3.77806071e+01  2.00000000e+00 ...  1.00000000e+00
   1.00000000e+00  0.00000000e+00]
 [-1.22394926e+02  3.77382115e+01  2.00000000e+00 ...  1.00000000e+00
   0.00000000e+00  1.00000000e+00]]
-------------------------------------------------------------------------
['WARRANTS' 'OTHER OFFENSES' 'OTHER OFFENSES' ... 'LARCENY/THEFT'
 'VANDALISM' 'FORGERY/COUNTERFEITING']
-------------------------------------------------------------------------
[[-1.22399588e+02  3.77350510e+01  0.